In [1]:
import numpy as np
import pandas as pd
from fbprophet import Prophet

In [2]:
train=pd.read_csv("../data/train.csv",low_memory=False)
store=pd.read_csv("../data/store.csv",low_memory=False)

train_store=pd.merge(train,store,how='inner',on='Store')
train_store = train_store[(train_store['Open']!=0)&(train_store['Sales']!=0)]
train_store.fillna(0,inplace=True)

In [3]:
#sales for store types

s_type='a'
train_store['Date']=pd.to_datetime(train_store['Date'])
sales = train_store[train_store['StoreType']==s_type].loc[:,['Date','Sales']]
sales.sort_values('Date',inplace=True)
sales.set_index('Date')

,Sales
Date,
2013-01-01,2907
2013-01-02,20642
2013-01-02,4389
2013-01-02,5578
2013-01-02,7446
...,...
2015-07-31,10054
2015-07-31,6370
2015-07-31,7763


In [4]:
sales.rename(columns={'Date':'ds','Sales':'y'},inplace=True)
sales.sample()

,ds,y
995936,2014-06-11,9792


**Model Holidays**
* Create holidays dataframe

In [5]:
state_dates = train_store[(train_store.StateHoliday == 'a') | (train_store.StateHoliday == 'b') & 
                          (train_store.StateHoliday == 'c')].loc[:, 'Date'].values
school_dates = train_store[train_store.SchoolHoliday == 1].loc[:, 'Date'].values

state_holiday = pd.DataFrame({'ds': pd.to_datetime(state_dates),'holiday': 'state_holiday'})
school_holiday = pd.DataFrame({'ds': pd.to_datetime(school_dates),'holiday': 'state_holiday'})

df = pd.concat((state_holiday, school_holiday))
df.sample()

,ds,holiday
87328,2014-08-05,state_holiday


**Forecast 6 weeks into the future**

In [6]:
model = Prophet(holidays = df)
model.fit(sales)
#6weeks = 42 days
dates= model.make_future_dataframe(periods = 42)
forecast = model.predict(dates)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [7]:
forecast.tail(10)

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,holidays,...,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
948,2015-09-02,6234.749700,2356.870789,9829.312378,6186.638832,6279.356544,-215.306666,-215.306666,-215.306666,0.0,...,409.678898,409.678898,409.678898,-624.985564,-624.985564,-624.985564,0.0,0.0,0.0,6019.443034
949,2015-09-03,6234.531504,2136.563308,9800.989646,6183.448251,6281.354435,-223.570021,-223.570021,-223.570021,0.0,...,424.540939,424.540939,424.540939,-648.110961,-648.110961,-648.110961,0.0,0.0,0.0,6010.961483
950,2015-09-04,6234.313308,2365.004467,9969.810148,6178.339856,6284.663763,-107.197475,-107.197475,-107.197475,0.0,...,558.333132,558.333132,558.333132,-665.530606,-665.530606,-665.530606,0.0,0.0,0.0,6127.115833
951,2015-09-05,6234.095112,1079.864251,8838.855150,6174.186774,6288.809682,-1535.273736,-1535.273736,-1535.273736,0.0,...,-858.818979,-858.818979,-858.818979,-676.454756,-676.454756,-676.454756,0.0,0.0,0.0,4698.821376
952,2015-09-06,6233.876915,-2134.647409,6268.801369,6171.396695,6291.203892,-3987.610372,-3987.610372,-3987.610372,0.0,...,-3307.378502,-3307.378502,-3307.378502,-680.231870,-680.231870,-680.231870,0.0,0.0,0.0,2246.266543
953,2015-09-07,6233.658719,3341.836757,11068.566969,6170.574756,6297.653934,1240.238948,1240.238948,1240.238948,0.0,...,1916.610962,1916.610962,1916.610962,-676.372013,-676.372013,-676.372013,0.0,0.0,0.0,7473.897668
954,2015-09-08,6233.440523,2464.856947,10717.863597,6167.104524,6301.919107,192.467198,192.467198,192.467198,0.0,...,857.033550,857.033550,857.033550,-664.566351,-664.566351,-664.566351,0.0,0.0,0.0,6425.907721
955,2015-09-09,6233.222327,1957.750338,9963.922360,6165.089830,6306.100151,-235.023205,-235.023205,-235.023205,0.0,...,409.678898,409.678898,409.678898,-644.702103,-644.702103,-644.702103,0.0,0.0,0.0,5998.199122
956,2015-09-10,6233.004131,2222.665052,9948.878891,6162.352247,6310.742554,-192.331468,-192.331468,-192.331468,0.0,...,424.540939,424.540939,424.540939,-616.872407,-616.872407,-616.872407,0.0,0.0,0.0,6040.672663
957,2015-09-11,6232.785935,2268.868905,9807.425631,6156.039524,6315.994511,-23.047586,-23.047586,-23.047586,0.0,...,558.333132,558.333132,558.333132,-581.380718,-581.380718,-581.380718,0.0,0.0,0.0,6209.738348
